# Get Battery Data

In [ ]:
import pymatgen as mg
import numpy as np
import pandas as pd
import xenonpy as xn
import numpy as np
from retrying import retry
from xenonpy.descriptor import Compositions
from xenonpy.datatools import preset
from pymatgen import MPRester

In [ ]:
##Takes in a str of battery id or formula to return battery data##
#@retry(wait_random_min=1000, wait_random_max=2000)
def get_battery_data(self, formula_or_batt_id):
    """Returns batteries from a batt id or formula.

    Examples:
        get_battery("mp-300585433")
        get_battery("LiFePO4")
    """
    return mpr._make_request('/battery/%s' % formula_or_batt_id)

In [ ]:
##Take the string of the element name to get the material ID of most stable element, its crystal structure##
#@retry(wait_random_min=1000, wait_random_max=2000)
def get_stable_element_id(element):
    
    ##Get data of all available data with element string##
    with MPRester("aQawb0MSciQagC4thO1") as mpr:
        results1 = mpr.get_data(element)
    
        ##Obtain list of material_id as string##
        df1 = pd.DataFrame(results1)
        materials_id = [None for i in range(len(df1))]
        for i in range(len(df1)):
            materials_id[i] = df1['material_id'][i]
        
        ##For list of bcc elements, remove material_id that are not bcc##
        bcc = ['Li', 'Na', 'K', 'Rb', 'Cs', 'Fr', 'Ba', 'Ra', 'V', 'Cr', 'Mn', 
               'Fe', "Nb", 'Mo', 'Ta', 'W', 'Db', 'Sg', 'Ds', 'Rg', 'Cn']
        for i in range (len(bcc)):
            count = 0
            if element == bcc[i]:
                count = 1
                for j in range (len(materials_id)):
                    structure = mpr.get_structure_by_material_id(materials_id[j])
                    crystal_structure = mg.analysis.local_env.site_is_of_motif_type(structure, 0)
                    if crystal_structure != 'bcc':
                        materials_id[j] = 0
            if count == 1:
                break
        
        materials_id = [i for i in materials_id if i != 0]
        #print(materials_id)
        
        ##Obtain list of e_above_hull for corresponding material_id##
        criteria = {'material_id' : {'$in' : materials_id}}
        properties = ['material_id', 'e_above_hull']
        results2 = mpr.query(criteria, properties)
        #print(results2)
                
        ##Iterate through results to get material_id with lowest e_above_hull = 0##
        df2 = pd.DataFrame(results2)
        e_above_hull_min = 9999999999
        #print(df2.head())
        for i in range(len(df2)):
            if df2['e_above_hull'][i] < e_above_hull_min:
                results3 = df2['material_id'][i]  
                e_above_hull_min = df2['e_above_hull'][i]
            if e_above_hull_min == 0:
                break
            break
        
        stable_structure = mpr.get_structure_by_material_id(results3)
        stable_crystal_structure = mg.analysis.local_env.site_is_of_motif_type(stable_structure, 0)
        
    return results3, stable_crystal_structure

In [ ]:
##Take the string of element name and return the necessary features stated in lit review"
#@retry(wait_random_min=1000, wait_random_max=2000)
def element_properties(element):
    
    ##Create empty dataframe##
    empty = pd.DataFrame()
    
    ##Obtain stable material_id##
    material_id, stable_crystal_structure = get_stable_element_id(element)
    #print(stable_crystal_structure)
    
    ##Obtain data from the most stable material_id##
    with MPRester("aQawb0MSciQagC4thO1") as mpr:
        results = mpr.get_data(material_id)
    #print(results)
    stable_data = pd.DataFrame(results)
    #print(stable_data)
    
    ##Obtain data from Xenonpy preset data##
    xenonpy_data = pd.DataFrame(preset.elements_completed)
    
    ##Append atomic_volume##
    atomic_volume = xenonpy_data.loc[[element], ['atomic_volume']]
    empty = pd.concat([empty, atomic_volume], axis = 1)
    
    ##Append atomic_number##
    atomic_number = mg.core.periodic_table.Element(element).Z
    empty['atomic_number'] = atomic_number
    
    ##Append atomic_weight##
    atomic_weight = mg.core.periodic_table.Element(element).atomic_mass
    empty['atomic_weight'] = atomic_weight
    
    ##Append covalent_radius##
    covalent_radius = mg.analysis.molecule_structure_comparator.CovalentRadius.radius
    empty['covalent_radius'] = covalent_radius[element]
    
    ##Append oxidation_states##
    oxidation_states = mg.core.periodic_table.Element(element).oxidation_states
    empty['oxidation_states'] = len(oxidation_states)
    
    ##Append boiling_point##
    boiling_point = mg.core.periodic_table.Element(element).boiling_point
    empty['boiling_temperature'] = boiling_point
    
    #Append column_number##
    column_number = mg.core.periodic_table.Element(element).group
    empty['column_number'] = column_number
    
    ##Append row_number##
    row_number = mg.core.periodic_table.Element(element).row
    empty['row_number'] = row_number
    
    ##Append first_ion_en##
    first_ion_en = xenonpy_data.loc[[element], ['first_ion_en']]
    empty = pd.concat([empty, first_ion_en], axis = 1)
    
    ##Append spacegroup_number##
    spacegroup = pd.Series(stable_data['spacegroup'][0])
    spacegroup_number = spacegroup['number']
    empty['spacegroup_number'] = spacegroup_number
    
    ##Append density##
    density = mg.core.periodic_table.Element(element).density_of_solid
    empty['density'] = density
    
    ##Append mendeleev_number##
    mendeleev_number = mg.core.periodic_table.Element(element).mendeleev_no
    empty['mendeleev_number'] = mendeleev_number
    
    ##Append icsd_volume##
    icsd_volume = xenonpy_data.loc[[element], ['icsd_volume']]
    empty = pd.concat([empty, icsd_volume], axis = 1)
    
    ##Append Polarizability##
    Polarizability = xenonpy_data.loc[[element], ['Polarizability']]
    empty = pd.concat([empty, Polarizability], axis = 1)
    
    ##Append bcc_energy##
    if stable_crystal_structure == 'bcc':
        empty['bcc_energy'] = stable_data['energy'][0]
    else:
        empty['bcc_energy'] = np.nan
    
    ##Append bbc_effective_latcnt
    if stable_crystal_structure == 'bcc':
        bbc_effective_latcnt = xenonpy_data.loc[[element], ['lattice_constant']]
        empty = pd.concat([empty, bbc_effective_latcnt], axis = 1)
        empty = empty.rename(columns = {'lattice_constant': 'bbc_effective_latcnt'})
    else:
        empty['bbc_effective_latcnt'] = np.nan
    
    ##Append bcc_fermi##
    if stable_crystal_structure == 'bcc':
        with MPRester("aQawb0MSciQagC4thO1") as mpr:
            stable_bandstructure = mpr.get_bandstructure_by_material_id(material_id)
        if stable_bandstructure is None:
            empty['bbc_fermi'] = np.nan
        else:
            efermi = stable_bandstructure.efermi
            empty['bbc_fermi'] = efermi
    else:
        empty['bbc_fermi'] = np.nan
    
    ##Append bcc_mag_mom##
    if stable_crystal_structure == 'bcc':
        bcc_mag_mom = stable_data['total_magnetization'][0]
        empty['bcc_mag_mom'] = bcc_mag_mom
    else:
        empty['bcc_mag_mom'] = np.nan
    
    ##Append bcc_volume_pa##
    if stable_crystal_structure == 'bcc':
        bcc_volume_pa = stable_data['volume'][0]
        empty['bcc_volume_pa'] = bcc_volume_pa
    else:
        empty['bcc_volume_pa'] = np.nan
    
    ##Append bcc_volume_padiff##
    if stable_crystal_structure == 'bcc':
        bcc_volume_padiff = abs(stable_data['volume'][0] - atomic_volume)
        empty['bcc_volume_padiff'] = bcc_volume_padiff
    else:
        empty['bcc_volume_padiff'] = np.nan
    
    ##Append heat_capacity_mass, heat_capacity_molar, gs_bandgap, gs_energy,##
    ##gs_est_bcc_latcnt, gs_est_fcc_latcnt, gs_mag_moment',gs_volume_per##
    heat_and_gs = xenonpy_data.loc[[element], ['heat_capacity_mass', 'heat_capacity_molar', 'gs_bandgap',
                  'gs_energy', 'gs_est_bcc_latcnt', 'gs_est_fcc_latcnt', 'gs_mag_moment', 'gs_volume_per']]
    empty = pd.concat([empty, heat_and_gs], axis = 1)
    
    ##Append is_metal, is_nonmetal, is_metalloid, is_alkali##
    is_alkali = mg.core.periodic_table.Element(element).is_alkali
    is_alkaline = mg.core.periodic_table.Element(element).is_alkaline
    is_transition_metal = mg.core.periodic_table.Element(element).is_transition_metal
    is_post_transition_metal = mg.core.periodic_table.Element(element).is_post_transition_metal
    is_rare_earth_metal = mg.core.periodic_table.Element(element).is_rare_earth_metal
    is_metalloid = mg.core.periodic_table.Element(element).is_metalloid
    
    if is_alkali == 1 or is_alkaline == 1 or is_transition_metal == 1 or is_post_transition_metal == 1 or is_rare_earth_metal == 1:
        empty['is_metal'] = 1
        empty['is_nonmetal'] = 0
    else:
        empty['is_metal'] = 0
        empty['is_nonmetal'] = 1
    
    empty['is_metalloid'] = is_metalloid
    empty['is_alkali'] = is_alkali
    
    ##Append is_d-block, is_f-block##
    block = mg.core.periodic_table.Element(element).block
    if block == 'd':
        empty['is_d-block'] = 1
        empty['is_f-block'] = 0
    elif block == 'f':
        empty['is_d-block'] = 0
        empty['is_f-block'] = 1
    else:
        empty['is_d-block'] = 0
        empty['is_f-block'] = 0
        
    ##Append num_valance, num_unfilled, num_s_valence, num_s_unfilled, num_p_valence, num_p_unfilled, 
    ##num_d_valence, num_d_unfilled, num_f_valence, num_f_unfilled##
    valence = xenonpy_data.loc[[element], ['num_valance', 'num_unfilled', 'num_s_valence',  'num_s_unfilled',
              'num_p_valence', 'num_p_unfilled', 'num_d_valence', 'num_d_unfilled', 'num_f_valence', 'num_f_unfilled']]
    empty = pd.concat([empty, valence], axis = 1)
    
    return empty

In [ ]:
##Takes in element data process it to give fraction weighted average and weighted variance##
#@retry(wait_random_min=1000, wait_random_max=2000)
def get_weighted_data(element_data, full_composition):
    
    ##Create empty dataframe##
    empty = pd.DataFrame()
    
    ##Find total_num_elements##
    total_num_elements = 0 
    for element in full_composition.index.values:
        total_num_elements += full_composition[element]
    #print(total_num_elements)
    
    ##Find the weighted_avg for each element and append to empty##
    for column in element_data.columns:
        avg = 0 
        for index in element_data.index.values:
            if np.isnan(element_data[column][index]) == 1:
                continue
            else:
                avg += element_data[column][index] *  full_composition[index]
        empty['avg: ' + column] = [avg / total_num_elements]
    #print(empty)
    
    ##Find the weighted_var for each element and append to empty##
    for column in element_data.columns:
        var = 0 
        for index in element_data.index.values:
            if np.isnan(element_data[column][index]) == 1:
                continue
            else:
                var += abs((element_data[column][index] - empty['avg: ' + column][0])) * full_composition[index]
        empty['var: ' + column] = var / total_num_elements
    
    ##Reset index##
    empty = empty.reset_index(drop = True)
    
    return empty

In [ ]:
##Takes in min/max composition and return L^p norms for p = 0, 2, 3, 5, 7, 10##
#@retry(wait_random_min=1000, wait_random_max=2000)
def get_lp_norms(full_composition, frac_type):
    
    ##Create empty dataframe##
    empty = pd.DataFrame()
    
    ##Find total_num_elements##
    total_num_elements = 0 
    for element in full_composition.index.values:
        total_num_elements += full_composition[element]
    #print(total_num_elements)
    empty[frac_type + "_lp_norm_" + "0"] = [total_num_elements]
    
    ##Generate list of norms##
    norm_values = [2, 3, 5, 7, 10]
    
    ##Get lp norms##
    for norm in norm_values:
        lp_norm = 0
        for element in full_composition.index.values:
            lp_norm += pow(full_composition[element] / total_num_elements, norm)
        empty[frac_type + "_lp_norm_" + str(norm)] = pow(lp_norm, 1 / norm)
    
    ##Reset index##
    empty = empty.reset_index(drop = True)
    
    return empty

In [ ]:
##Take in battery data and returns dataframe with all features and label##
#@retry(wait_random_min=1000, wait_random_max=2000)
def get_data(results):
    
    ##Create empty dataframe##
    empty = pd.DataFrame()
    
    ##Get the first layer of unique useful features to append to empty##
    df = pd.DataFrame(results)
    df_useful1 = df.loc[:, ['min_frac', 'max_frac','nsteps','numsites', 'type']]    
    empty = pd.concat([empty, df_useful1], axis = 1)
    
    ##Get the second layer of unique useful features from "spacegroup" to append to empty##
    df_spacegroup = pd.Series(df['spacegroup'][0])
    empty['spacegroup_number'] = df_spacegroup['number']
    empty['spacegroup_hall_number'] = df_spacegroup['hall_number']
#     for keys in df_spacegroup.keys():
#         if type(df_spacegroup[keys]) is str:
#             if len(df_spacegroup[keys]) == 0:
#                 empty['spacegroup_' + keys] = np.nan
#             else:
#                 empty['spacegroup_' + keys] = df_spacegroup[keys]
#         else:
#             empty['spacegroup_' + keys] = df_spacegroup[keys]

    ##Get the second layer of unique useful featues from "adj_pairs" to append to empty##
    useful_features = ['capacity_grav', 'capacity_vol', 'energy_grav', 'energy_vol', 'fracA_charge', 
                       'fracA_discharge', 'stability_charge', 'stability_discharge', 'working_ion']
    df_adj_pairs = pd.DataFrame(df['adj_pairs'][0])
    for keys in useful_features:
        empty[keys] = df_adj_pairs[keys][0] 
    
    ##Use min_frac and max_frac to determine working_ion composition at low and high concentrations##
    reduced_cell_composition = pd.Series(df['reduced_cell_composition'][0])
    no_of_reduced_elements = 0
    for element in reduced_cell_composition.index.values:
        no_of_reduced_elements += reduced_cell_composition[element]
    no_of_working_ion_max = round((empty['max_frac'][0] * no_of_reduced_elements) / (1 - empty['max_frac'][0]))
    no_of_working_ion_min = round((empty['min_frac'][0] * no_of_reduced_elements) / (1 - empty['min_frac'][0]))
    working_ion_composition_max = pd.Series(no_of_working_ion_max, empty['working_ion'])
    working_ion_composition_min = pd.Series(no_of_working_ion_min, empty['working_ion'])
    full_composition_max = reduced_cell_composition.append(working_ion_composition_max)
    full_composition_min = reduced_cell_composition.append(working_ion_composition_min)

    ##Get stoichiometric attributes##
    lp_norm_max = get_lp_norms(full_composition_max, frac_type = 'max')
    lp_norm_min = get_lp_norms(full_composition_max, frac_type = 'min')
    empty = pd.concat([empty, lp_norm_max], axis = 1)
    empty = pd.concat([empty, lp_norm_min], axis = 1)
    
    ##Create featured dataframe from elements properties and append to empty##
    element_empty = pd.DataFrame()
    for index in full_composition_max.index.values:
        element_empty = pd.concat([element_empty, element_properties(index)], axis = 0)
    element_weighted = get_weighted_data(element_empty, full_composition_max)
    empty = pd.concat([empty, element_weighted], axis = 1)

    ##Append the labels: average_voltage## 
    df_labels = df.loc[:, ['average_voltage']]
#     df_labels = df_labels.abs()
#     df_labels = df_labels.rename(columns = {'average_voltage': 'abs_average_voltage'})
    empty = pd.concat([empty, df_labels], axis = 1)
    
    return empty, empty.columns

In [ ]:
with MPRester("aQawb0MSciQagC4thO1") as mpr:
    MPRester.get_battery_data = get_battery_data
    results = mpr.get_battery_data('mp-1045691_Mg')
results

In [ ]:
empty, empty_columns = get_data(results)
print(empty)
print(list(empty_columns))

In [ ]:
##Loads dataset for all batteries in battery explorer##
@retry(wait_random_min=1000, wait_random_max=2000)
def get_full_dataset():
    
    ##Obtain list of all battery ids##
    with MPRester("aQawb0MSciQagC4thO1") as mpr:
        battery_list = mpr._make_request('/battery/all_ids')
    
    ##Iterate over all possible battery ids in materials project##
    MPRester.get_battery_data = get_battery_data
    count = 0
    num = 0 
    for battery_id in battery_list:
        with MPRester("aQawb0MSciQagC4thO1") as mpr:
            results = mpr.get_battery_data(battery_id)
        if len(results) == 0 :
            print(battery_id + 'has no data from get_battery_data')
            continue
        elif count == 0:
            data, columns = get_data(results)
            empty = pd.DataFrame(columns = columns)
            if data.empty == 1:
                print(battery_id + 'has no data from get_data')
                continue
            else:
                print(battery_id + ' this is data number: ' + str(num))
                empty = pd.concat([empty, data], axis = 0)
                empty = empty.rename(index = {0: battery_id})
                count += 1
                num += 1
        else:
            data, _ = get_data(results)
            if data.empty == 1:
                print(battery_id + 'has no data from get_data')
                continue
            else:
                print(battery_id + ' this is data number: ' + str(num))
                empty = pd.concat([empty, data], axis = 0)
                empty = empty.rename(index = {0: battery_id})
                num += 1
    
    return empty

In [ ]:
data = get_full_dataset()
data

In [ ]:
data.to_csv('battery_dataset.csv')

In [ ]:
with MPRester("aQawb0MSciQagC4thO1") as mpr:
    battery_list = mpr._make_request('/battery/all_ids')
battery_list

In [ ]:
##Create empty dataframe with 3 key features##
expt_electrodes = pd.DataFrame(columns = ['min_frac', 'max_frac', 'spacegroup_number'])

##Add data for NaMnO2 from Bilaud##
expt_electrodes.loc['NaMnO2', 'min_frac'] = 0.236 / (0.236 + 1 + 2)
expt_electrodes.loc['NaMnO2', 'max_frac'] = 0.92 / (0.92 + 1 + 2)
expt_electrodes.loc['NaMnO2', 'spacegroup_number'] = 59
expt_electrodes.loc['NaMnO2', 'type'] = 'intercalation' 
expt_electrodes.loc['NaMnO2', 'working_ion'] = 'Na'
expt_electrodes.loc['NaMnO2', 'average_voltage'] = 2.75

##Add data for NaCoO2 from Ong##
expt_electrodes.loc['NaCoO2', 'min_frac'] = 0.5 / (0.5 + 1 + 2)
expt_electrodes.loc['NaCoO2', 'max_frac'] = 1 / (1 + 1 + 2)
expt_electrodes.loc['NaCoO2', 'spacegroup_number'] = 166
expt_electrodes.loc['NaCoO2', 'type'] = 'intercalation' 
expt_electrodes.loc['NaCoO2', 'working_ion'] = 'Na'
expt_electrodes.loc['NaCoO2', 'average_voltage'] = 2.80

##Add data for NaTiO2 from Ong##
expt_electrodes.loc['NaTiO2', 'min_frac'] = 0.7 / (0.7 + 1 + 2)
expt_electrodes.loc['NaTiO2', 'max_frac'] = 1 / (1 + 1 + 2)
expt_electrodes.loc['NaTiO2', 'spacegroup_number'] = 166
expt_electrodes.loc['NaTiO2', 'type'] = 'intercalation' 
expt_electrodes.loc['NaTiO2', 'working_ion'] = 'Na'
expt_electrodes.loc['NaTiO2', 'average_voltage'] = 1.50

##Add data for NaNiO2 from Ong##
expt_electrodes.loc['NaNiO2', 'min_frac'] = 0.8 / (0.8 + 1 + 2)
expt_electrodes.loc['NaNiO2', 'max_frac'] = 1 / (1 + 1 + 2)
expt_electrodes.loc['NaNiO2', 'spacegroup_number'] = 166
expt_electrodes.loc['NaNiO2', 'type'] = 'intercalation' 
expt_electrodes.loc['NaNiO2', 'working_ion'] = 'Na'
expt_electrodes.loc['NaNiO2', 'average_voltage'] = 3.00

##Add data for LiCoO2 from Ong##
expt_electrodes.loc['LiCoO2', 'min_frac'] = 0
expt_electrodes.loc['LiCoO2', 'max_frac'] = 1 / (1 + 1 + 2)
expt_electrodes.loc['LiCoO2', 'spacegroup_number'] = 166
expt_electrodes.loc['LiCoO2', 'type'] = 'intercalation' 
expt_electrodes.loc['LiCoO2', 'working_ion'] = 'Li'
expt_electrodes.loc['LiCoO2', 'average_voltage'] = 4.10

##Add data for NaFePO4 from Ong##
expt_electrodes.loc['NaFePO4', 'min_frac'] = 0
expt_electrodes.loc['NaFePO4', 'max_frac'] = 0.7 / (0.7 + 1 + 1 + 4)
expt_electrodes.loc['NaFePO4', 'spacegroup_number'] = 62
expt_electrodes.loc['NaFePO4', 'type'] = 'intercalation' 
expt_electrodes.loc['NaFePO4', 'working_ion'] = 'Na'
expt_electrodes.loc['NaFePO4', 'average_voltage'] = 3.00

##Add data for Na4MnV(PO4)3 from Nisar##
expt_electrodes.loc['Na4MnV(PO4)3', 'min_frac'] = 3 / (4 + 1 + 1 + 3 + 12)
expt_electrodes.loc['Na4MnV(PO4)3', 'max_frac'] = 4 / (4 + 1 + 1 + 3 + 12)
expt_electrodes.loc['Na4MnV(PO4)3', 'spacegroup_number'] = 167
expt_electrodes.loc['Na4MnV(PO4)3', 'type'] = 'intercalation' 
expt_electrodes.loc['Na4MnV(PO4)3', 'working_ion'] = 'Na'
expt_electrodes.loc['Na4MnV(PO4)3', 'average_voltage'] = 3.00

##Add data for LiFePO4 from Ong##
expt_electrodes.loc['LiFePO4', 'min_frac'] = 0
expt_electrodes.loc['LiFePO4', 'max_frac'] = 0.6 / (0.6 + 1 + 1 + 4)
expt_electrodes.loc['LiFePO4', 'spacegroup_number'] = 62
expt_electrodes.loc['LiFePO4', 'type'] = 'intercalation' 
expt_electrodes.loc['LiFePO4', 'working_ion'] = 'Li'
expt_electrodes.loc['LiFePO4', 'average_voltage'] = 3.50

##Add data for LiNiO2 from Ong##
expt_electrodes.loc['LiNiO2', 'min_frac'] = 0
expt_electrodes.loc['LiNiO2', 'max_frac'] = 1 / (1 + 1 + 2)
expt_electrodes.loc['LiNiO2', 'spacegroup_number'] = 166
expt_electrodes.loc['LiNiO2', 'type'] = 'intercalation' 
expt_electrodes.loc['LiNiO2', 'working_ion'] = 'Li'
expt_electrodes.loc['LiNiO2', 'average_voltage'] = 3.85

##Add data for NaFe0.5Co0.5O2 from Yoshida##
expt_electrodes.loc['NaFe0.5Co0.5O2', 'min_frac'] = 1 / (1 + 0.5 + 0.5 + 2)
expt_electrodes.loc['NaFe0.5Co0.5O2', 'max_frac'] = 1 / (1 + 0.5 + 0.5 + 2)
expt_electrodes.loc['NaFe0.5Co0.5O2', 'spacegroup_number'] = 166
expt_electrodes.loc['NaFe0.5Co0.5O2', 'type'] = 'intercalation' 
expt_electrodes.loc['NaFe0.5Co0.5O2', 'working_ion'] = 'Na'
expt_electrodes.loc['NaFe0.5Co0.5O2', 'average_voltage'] = 3.14

##Add data for K1.6Na2Mn3O7 from Sada##
expt_electrodes.loc['K1.6Na2Mn3O7', 'min_frac'] = 0
expt_electrodes.loc['K1.6Na2Mn3O7', 'max_frac'] = 1.6 / (1.6 + 2 + 3 + 7)
expt_electrodes.loc['K1.6Na2Mn3O7', 'spacegroup_number'] = 2
expt_electrodes.loc['K1.6Na2Mn3O7', 'type'] = 'intercalation' 
expt_electrodes.loc['K1.6Na2Mn3O7', 'working_ion'] = 'K'
expt_electrodes.loc['K1.6Na2Mn3O7', 'average_voltage'] = 2.20

##Add data for Mg2Mo6S8 from Canepa##
expt_electrodes.loc['Mg2Mo6S8', 'min_frac'] = 0
expt_electrodes.loc['Mg2Mo6S8', 'max_frac'] = 2 / (2 + 6 + 8)
expt_electrodes.loc['Mg2Mo6S8', 'spacegroup_number'] = 148
expt_electrodes.loc['Mg2Mo6S8', 'type'] = 'intercalation' 
expt_electrodes.loc['Mg2Mo6S8', 'working_ion'] = 'Mg'
expt_electrodes.loc['Mg2Mo6S8', 'average_voltage'] = 1.30

##Add data for Mg0.55TiSe2 from Gu##
expt_electrodes.loc['Mg0.55TiSe2', 'min_frac'] = 0.05 / (0.05 + 1 + 2)
expt_electrodes.loc['Mg0.55TiSe2', 'max_frac'] = 0.48 / (0.48 + 1 + 2)
expt_electrodes.loc['Mg0.55TiSe2', 'spacegroup_number'] = 164
expt_electrodes.loc['Mg0.55TiSe2', 'type'] = 'intercalation' 
expt_electrodes.loc['Mg0.55TiSe2', 'working_ion'] = 'Mg'
expt_electrodes.loc['Mg0.55TiSe2', 'average_voltage'] = 1.45

##Add data for MgMoO3 from Gershinky##
expt_electrodes.loc['MgMoO3', 'min_frac'] = 0
expt_electrodes.loc['MgMoO3', 'max_frac'] = 0.5 / (0.5 + 1 + 3)
expt_electrodes.loc['MgMoO3', 'spacegroup_number'] = 7
expt_electrodes.loc['MgMoO3', 'type'] = 'intercalation' 
expt_electrodes.loc['MgMoO3', 'working_ion'] = 'Mg'
expt_electrodes.loc['MgMoO3', 'average_voltage'] = 2.25

expt_electrodes

In [ ]:
##Composition of min frac and max frac experimental electrodes##
expt_electrodes_min_frac_comp = [{'NaMnO2' : {'Na' : 0.236, 'Mn' : 1, 'O': 2}, 
                                'NaCoO2' : {'Na' : 0, 'Co' : 1, 'O': 2},
                                'NaTiO2' : {'Na' : 0, 'Ti' : 1, 'O': 2},
                                'NaNiO2' : {'Na' : 0, 'Ni' : 1, 'O': 2},
                                'LiCoO2' : {'Li' : 0, 'Co' : 1, 'O': 2},
                                'NaFePO4' : {'Na' : 0, 'Fe' : 1, 'P' : 1, 'O': 4},
                                'Na4MnV(PO4)3' : {'Na' : 3, 'Mn' : 1, 'V' : 1, 'P' : 3, 'O': 12},
                                'LiFePO4' : {'Li' : 0, 'Fe' : 1, 'P' : 1, 'O': 4},
                                'LiNiO2' : {'Li' : 0, 'Ni' : 1, 'O': 2},
                                'NaFe0.5Co0.5O2' : {'Na' : 0, 'Fe' : 0.5, 'Co' : 0.5, 'O': 2},
                                'K1.6Na2Mn3O7' : {'K' : 0, 'Na' : 2, 'Mn' : 3, 'O': 7},
                                'Mg2Mo6S8' : {'Mg' : 0, 'Mo' : 6, 'S': 8},
                                'Mg0.55TiSe2' : {'Mg' : 0, 'Ti' : 1, 'Se': 2},
                                'MgMoO3' : {'Mg' : 0, 'Mo' : 1, 'O': 3}}]
expt_electrodes_max_frac_comp = [{'NaMnO2' : {'Na' : 1, 'Mn' : 1, 'O': 2}, 
                                'NaCoO2' : {'Na' : 1, 'Co' : 1, 'O': 2},
                                'NaTiO2' : {'Na' : 1, 'Ti' : 1, 'O': 2},
                                'NaNiO2' : {'Na' : 1, 'Ni' : 1, 'O': 2},
                                'LiCoO2' : {'Li' : 1, 'Co' : 1, 'O': 2},
                                'NaFePO4' : {'Na' : 1, 'Fe' : 1, 'P' : 1, 'O': 4},
                                'Na4MnV(PO4)3' : {'Na' : 4, 'Mn' : 1, 'V' : 1, 'P' : 3, 'O': 12},
                                'LiFePO4' : {'Li' : 1, 'Fe' : 1, 'P' : 1, 'O': 4},
                                'LiNiO2' : {'Li' : 1, 'Ni' : 1, 'O': 2},
                                'NaFe0.5Co0.5O2' : {'Na' : 1, 'Fe' : 0.5, 'Co' : 0.5, 'O': 2},
                                'K1.6Na2Mn3O7' : {'K' : 1.6, 'Na' : 2, 'Mn' : 3, 'O': 7},
                                'Mg2Mo6S8' : {'Mg' : 2, 'Mo' : 6, 'S': 8},
                                'Mg0.55TiSe2' : {'Mg' : 0.55, 'Ti' : 1, 'Se': 2},
                                'MgMoO3' : {'Mg' : 1, 'Mo' : 1, 'O': 3}}]

##Turn data into pandas data##
min_comp = pd.DataFrame(expt_electrodes_min_frac_comp)
max_comp = pd.DataFrame(expt_electrodes_max_frac_comp)

##Create empty database for electrodes elemental data and stoichiometric attributes##
electrodes_elementnstoic_data = pd.DataFrame()
count = 0

##Iterate over compositions of various electrodes, obtain weighted data and append to electrodes_element_data##
for electrode in max_comp.columns.values:
    
    ##Get stoichiometric data##
    print(electrode)
    min_comp_series = pd.Series(min_comp[electrode][0])
    electrodes_stoi_min = get_lp_norms(min_comp_series, frac_type = 'min')
    print(electrodes_stoi_min)
    max_comp_series = pd.Series(max_comp[electrode][0])
    electrodes_stoi_max = get_lp_norms(max_comp_series, frac_type = 'max')
    print(electrodes_stoi_max)
    lp_norm = pd.concat([electrodes_stoi_max, electrodes_stoi_min], axis = 1, join = "inner")
    print(lp_norm)
    
    ##Get weighted elemental data##
    single_electrode_data = pd.DataFrame()
    for index in max_comp[electrode][0]:
        print(index)
        single_electrode_data = pd.concat([single_electrode_data, element_properties(index)], axis = 0)
        print(single_electrode_data)
    element_weighted = get_weighted_data(single_electrode_data, max_comp_series)
    print(element_weighted)
    
    ##Add stoichiometric data and weighted element data##
    sum_data = pd.concat([lp_norm, element_weighted], axis = 1, join = "inner")
    sum_data.rename(index = {0 : electrode}, inplace = True)
    
    ##Add to elementnstoic##
    electrodes_elementnstoic_data = pd.concat([electrodes_elementnstoic_data, sum_data], axis = 0)
    print(electrodes_elementnstoic_data)
    
    
electrodes_elementnstoic_data

In [ ]:
##Append elemental data with initial electrode data##
expt_electrodes = pd.concat([expt_electrodes, electrodes_elementnstoic_data], axis = 1)
expt_electrodes

In [ ]:
##Save electrode data as csv##
expt_electrodes.to_csv('experimental_electrodes_data.csv')

In [ ]:
print(list(expt_electrodes.columns))